In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jithinanievarghese/drugs-side-effects-and-medical-condition")

print("Path to dataset files:", path)

100%|██████████| 1.17M/1.17M [00:00<00:00, 1.23MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/jithinanievarghese/drugs-side-effects-and-medical-condition/versions/1


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, r2_score

In [4]:
df=pd.read_csv('/content/drugs_side_effects_drugs_com.csv')
df.head()

,drug_name,medical_condition,side_effects,generic_name,drug_classes,brand_names,activity,rx_otc,pregnancy_category,csa,alcohol,related_drugs,medical_condition_description,rating,no_of_reviews,drug_link,medical_condition_url
0,doxycycline,Acne,"(hives, difficult breathing, swelling in your ...",doxycycline,"Miscellaneous antimalarials, Tetracyclines","Acticlate, Adoxa CK, Adoxa Pak, Adoxa TT, Alod...",87%,Rx,D,N,X,amoxicillin: https://www.drugs.com/amoxicillin...,Acne Other names: Acne Vulgaris; Blackheads; B...,6.8,760.0,https://www.drugs.com/doxycycline.html,https://www.drugs.com/condition/acne.html
1,spironolactone,Acne,hives ; difficulty breathing; swelling of your...,spironolactone,"Aldosterone receptor antagonists, Potassium-sp...","Aldactone, CaroSpir",82%,Rx,C,N,X,amlodipine: https://www.drugs.com/amlodipine.h...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.2,449.0,https://www.drugs.com/spironolactone.html,https://www.drugs.com/condition/acne.html
2,minocycline,Acne,"skin rash, fever, swollen glands, flu-like sym...",minocycline,Tetracyclines,"Dynacin, Minocin, Minolira, Solodyn, Ximino, V...",48%,Rx,D,N,NaN,amoxicillin: https://www.drugs.com/amoxicillin...,Acne Other names: Acne Vulgaris; Blackheads; B...,5.7,482.0,https://www.drugs.com/minocycline.html,https://www.drugs.com/condition/acne.html
3,Accutane,Acne,problems with your vision or hearing; muscle o...,isotretinoin (oral),"Miscellaneous antineoplastics, Miscellaneous u...",NaN,41%,Rx,X,N,X,doxycycline: https://www.drugs.com/doxycycline...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.9,623.0,https://www.drugs.com/accutane.html,https://www.drugs.com/condition/acne.html
4,clindamycin,Acne,hives ; difficult breathing; swelling of your ...,clindamycin topical,"Topical acne agents, Vaginal anti-infectives","Cleocin T, Clindacin ETZ, Clindacin P, Clindag...",39%,Rx,B,N,NaN,doxycycline: https://www.drugs.com/doxycycline...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.4,146.0,https://www.drugs.com/mtm/clindamycin-topical....,https://www.drugs.com/condition/acne.html


In [5]:
df['medical_condition'].unique()

array(['Acne', 'ADHD', 'AIDS/HIV', 'Allergies', "Alzheimer's", 'Angina',
       'Anxiety', 'Asthma', 'Bipolar Disorder', 'Bronchitis', 'Cancer',
       'Cholesterol', 'Colds & Flu', 'Constipation', 'COPD', 'Covid 19',
       'Depression', 'Diabetes (Type 1)', 'Diabetes (Type 2)', 'Diarrhea',
       'Eczema', 'Erectile Dysfunction', 'Gastrointestinal',
       'GERD (Heartburn)', 'Gout', 'Hair Loss', 'Hayfever', 'Herpes',
       'Hypertension', 'Hypothyroidism', 'IBD (Bowel)', 'Incontinence',
       'Insomnia', 'Menopause', 'Migraine', 'Osteoarthritis',
       'Osteoporosis', 'Pain', 'Pneumonia', 'Psoriasis',
       'Rheumatoid Arthritis', 'Schizophrenia', 'Seizures', 'Stroke',
       'Swine Flu', 'UTI', 'Weight Loss'], dtype=object)

In [6]:
df['side_effects'].unique()

array(['(hives, difficult breathing, swelling in your face or throat) or a severe skin reaction (fever, sore throat, burning in your eyes, skin pain, red or purple skin rash that spreads and causes blistering and peeling). Seek medical treatment if you have a serious drug reaction that can affect many parts of your body. Symptoms may include: skin rash, fever, swollen glands, flu-like symptoms, muscle aches, severe weakness, unusual bruising, or yellowing of your skin or eyes. This reaction may occur several weeks after you began using doxycycline. Doxycycline may cause serious side effects. Call your doctor at once if you have: severe stomach pain, diarrhea that is watery or bloody; throat irritation, trouble swallowing; chest pain, irregular heart rhythm, feeling short of breath; little or no urination; low white blood cell counts - fever, chills, swollen glands, body aches, weakness, pale skin, easy bruising or bleeding; severe headaches, ringing in your ears, dizziness, nausea, vis

In [7]:
df['drug_classes'].unique()

array(['Miscellaneous antimalarials, Tetracyclines',
       'Aldosterone receptor antagonists, Potassium-sparing diuretics',
       'Tetracyclines',
       'Miscellaneous antineoplastics, Miscellaneous uncategorized agents',
       'Topical acne agents, Vaginal anti-infectives',
       'Topical acne agents', 'Sulfonamides',
       'First generation cephalosporins',
       'Topical acne agents, Topical antipsoriatics', nan,
       'Topical acne agents, Topical antibiotics', 'Contraceptives',
       'Antacids, Laxatives', 'Miscellaneous uncategorized agents',
       'Topical acne agents, Topical anti-rosacea agents',
       'Topical acne agents, Topical keratolytics',
       'Mouth and throat products', 'Topical antibiotics',
       'Contraceptives, Sex hormone combinations',
       'Miscellaneous topical agents', 'Topical antipsoriatics',
       'CNS stimulants',
       'Adamantane antivirals, Dopaminergic antiparkinsonism agents',
       'Adrenergic uptake inhibitors for ADHD',
       

In [8]:
df=df.drop(columns=['related_drugs'])

In [9]:
filtered_df=df.drop(columns='drug_link')
filtered_df=df.drop(columns='medical_condition_url')

In [10]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df["generic_name"] = label_encoder.fit_transform(df["generic_name"])
df["medical_condition"] = label_encoder.fit_transform(df["medical_condition"])
df["pregnancy_category"] = label_encoder.fit_transform(df["pregnancy_category"])
df["side_effects"] = label_encoder.fit_transform(df["side_effects"])

In [11]:
## apply stemming
import nltk
from nltk.stem.porter import PorterStemmer
def stemmer(text):
    ps = nltk.PorterStemmer()
    text = " ".join([ps.stem(word) for word in text.split()])
    return text

In [12]:
df["medical_condition_description"] = df["medical_condition_description"].apply(stemmer)

In [13]:

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer

In [14]:
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer

stop_wr=set(stopwords.words('english'))
print(stop_wr)
#removing the stopwords
def removing_stopwords(text, is_lower_case=False):
    #Tokenization of text
    tokenizers=ToktokTokenizer()
    #Setting English stopwords
    tokens = tokenizers.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filter_tokens = [token for token in tokens if token not in stop_wr]
    else:
        filter_tokens = [token for token in tokens if token.lower() not in stop_wr]
    filtered_text = ' '.join(filter_tokens)
    return filtered_text

{'wasn', 'were', 'or', "they'll", "isn't", 'weren', 'before', 'who', 'whom', 'd', 'during', 'our', 'more', 'at', 'why', 'own', 'does', 'it', 'ourselves', 'do', 'will', 'o', 'how', "you're", 'i', 'myself', 'down', 'have', 'she', 'because', 'those', "i'd", "he'll", "they've", "we're", 'below', 'don', 'this', 'ain', 'for', 'couldn', "they're", 'himself', 'after', "i've", 'his', 'won', 'herself', 'wouldn', "you've", 'll', 'further', 'now', 'aren', 'doing', 'is', 'up', 'about', 'has', 'yourselves', 'into', 'where', 'out', "it'd", 'between', 'haven', 'her', "that'll", 'through', 'as', 'but', 'by', "she's", 'itself', 'both', 'here', 'some', 'very', 'yours', "mustn't", 'having', 'shouldn', "should've", 'the', "you'll", "you'd", 'above', 'ma', 'while', 'am', "don't", 'we', 'they', 'any', "i'll", "they'd", 'when', "didn't", 'just', 'had', 'an', "she'd", "wouldn't", 'then', 'you', 'what', 'its', 'once', 'been', "hadn't", 'and', 'again', 'same', 'to', 'if', 'off', 'are', 'did', 'with', 'was', 'in'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
df["medical_condition_description"] = df["medical_condition_description"].apply(removing_stopwords)

In [16]:
df.head()

,drug_name,medical_condition,side_effects,generic_name,drug_classes,brand_names,activity,rx_otc,pregnancy_category,csa,alcohol,medical_condition_description,rating,no_of_reviews,drug_link,medical_condition_url
0,doxycycline,2,15,641,"Miscellaneous antimalarials, Tetracyclines","Acticlate, Adoxa CK, Adoxa Pak, Adoxa TT, Alod...",87%,Rx,3,N,X,acn names : acn vulgaris ; blackheads ; breako...,6.8,760.0,https://www.drugs.com/doxycycline.html,https://www.drugs.com/condition/acne.html
1,spironolactone,2,1971,1269,"Aldosterone receptor antagonists, Potassium-sp...","Aldactone, CaroSpir",82%,Rx,2,N,X,acn names : acn vulgaris ; blackheads ; breako...,7.2,449.0,https://www.drugs.com/spironolactone.html,https://www.drugs.com/condition/acne.html
2,minocycline,2,2696,1033,Tetracyclines,"Dynacin, Minocin, Minolira, Solodyn, Ximino, V...",48%,Rx,3,N,NaN,acn names : acn vulgaris ; blackheads ; breako...,5.7,482.0,https://www.drugs.com/minocycline.html,https://www.drugs.com/condition/acne.html
3,Accutane,2,2569,902,"Miscellaneous antineoplastics, Miscellaneous u...",NaN,41%,Rx,5,N,X,acn names : acn vulgaris ; blackheads ; breako...,7.9,623.0,https://www.drugs.com/accutane.html,https://www.drugs.com/condition/acne.html
4,clindamycin,2,1259,504,"Topical acne agents, Vaginal anti-infectives","Cleocin T, Clindacin ETZ, Clindacin P, Clindag...",39%,Rx,1,N,NaN,acn names : acn vulgaris ; blackheads ; breako...,7.4,146.0,https://www.drugs.com/mtm/clindamycin-topical....,https://www.drugs.com/condition/acne.html


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming df is your DataFrame
tfidf_vectorizer = TfidfVectorizer(min_df=1, max_df=0.95, use_idf=True, ngram_range=(1, 3))

# Transform the textual data
tfidf_matrix = tfidf_vectorizer.fit_transform(df['medical_condition_description'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Concatenate the original DataFrame with the TF-IDF DataFrame
df = pd.concat([df, tfidf_df], axis=1)

# Drop the original 'medical_condition_description' column if needed
df.drop('medical_condition_description', axis=1, inplace=True)

In [18]:
df["side_effects"] = df['side_effects'].fillna('no')

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report

categorical_columns = df.select_dtypes(include=['object']).columns

# Convert categorical columns to numeric using one-hot encoding
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Handle missing values by filling them with the mean (or use another strategy)
df = df.fillna(df.mean())

# Features and target variable
X = df.drop('side_effects', axis=1)  # Features
y = df['side_effects']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = rf_classifier.score(X_test, y_test)
print('Accuracy: ', accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy:  0.04258943781942078
Classification Report:
               precision    recall  f1-score   support

           2       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         0
          20       0.00      0.00      0.00         1
          21       0.00      0.00      0.00         1
          28       0.00      0.00      0.00         0
          30       0.00      0.00      0.00         1
          35       0.00      0.00      0.00         1
          37       0.00      0.00      0.00         1
          38       0.00      0.00      0.00         0
          44       0.00      0.00      0.00         1
          45       0.00      0.00      0.00         1
          48       0.00      0.00      0.00         1
          49       0.00      0.00      0.00         0
          50       0.00   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [20]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

# Assuming the columns are named 'drug_classes', 'rx_otc', 'csa', 'alcohol', 'brand_names','side_effects', and 'medical_condition'
categorical_features = ['drug_classes', 'rx_otc', 'csa', 'alcohol', 'brand_names', 'side_effects']  # Adjust if column names are different
target_column = 'medical_condition'
data=pd.read_csv('/content/drugs_side_effects_drugs_com.csv')
# Select features and target
# Check the name of the side_effects column and if it is present
print(data.columns)
# Update based on the values that show up in the print statement
X = data[categorical_features]
y = data[target_column]

# Define the transformer for one-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features) # sparse=False for compatibility with RandomForest
    ])

# Create the pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100))
])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print("Classification Report:\n", classification_report(y_test, y_pred))

Index(['drug_name', 'medical_condition', 'side_effects', 'generic_name',
       'drug_classes', 'brand_names', 'activity', 'rx_otc',
       'pregnancy_category', 'csa', 'alcohol', 'related_drugs',
       'medical_condition_description', 'rating', 'no_of_reviews', 'drug_link',
       'medical_condition_url'],
      dtype='object')
Accuracy: 0.7802385008517888
Classification Report:
                       precision    recall  f1-score   support

                ADHD       1.00      0.77      0.87        13
            AIDS/HIV       0.90      0.95      0.93        20
                Acne       1.00      0.94      0.97        48
           Allergies       1.00      0.14      0.25         7
         Alzheimer's       0.88      0.88      0.88         8
              Angina       0.89      0.84      0.86        19
             Anxiety       0.00      0.00      0.00        12
              Asthma       0.30      0.30      0.30        10
    Bipolar Disorder       0.40      0.80      0.53     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
